In [25]:
from pandas import read_csv
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras import Model

from keras.models import Sequential
from keras.layers import Dense, Input, Attention
from keras.layers import LSTM
from keras.layers import Layer
from keras.layers import *
from keras.models import *
from keras import backend as K

from keras.layers import SimpleRNN, BatchNormalization, LeakyReLU
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor


In [26]:
def load_data():
    dataset = read_csv('solar_AL.txt', header=0, usecols = [5], engine='python')
    dataset[:] = numpy.nan_to_num(dataset)    
    return dataset.values

def create_dataset(dataset, look_back=1, look_ahead=0):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back - look_ahead-1):
        a = dataset[i:(i+look_back), 0]
        
        dataX.append(a)
        dataY.append(dataset[i + look_back + look_ahead, 0])
    return numpy.array(dataX), numpy.array(dataY)
load_data()

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [27]:
look_back = 1
look_ahead = 0
# Use multiple scalers for different columns / features
# 
dataset = load_data()
print(dataset)

scaler_train = MinMaxScaler(feature_range=(-1, 1))
scaler_test = MinMaxScaler(feature_range=(-1, 1))


train_size = int(len(dataset) * 0.80)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(train.shape, test.shape)

train = scaler_train.fit_transform(train)
test = scaler_test.fit_transform(test)

[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
(42047, 1) (10512, 1)


In [28]:
def make_model():
    model = Sequential()
    model.add(LSTM(4, input_shape=(1, look_back)))
    model.add(Dense(16, activation = 'tanh', kernel_initializer = 'he_normal'))
    model.add(Dense(8, activation = 'tanh', kernel_initializer = 'he_normal'))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error', optimizer='adam')
    

    return model

In [ ]:
gen_losses = []
disc_losses = []
for look_ahead in range(0, 50):
    trainX, trainY = create_dataset(train, look_back, look_ahead)
    testX, testY = create_dataset(test, look_back, look_ahead)
#     print(trainX.shape, [testX].shape, testY.shape)
    trainX = numpy.reshape(trainX, (trainX.shape[0], look_back, trainX.shape[1]))
    testX = numpy.reshape(testX, (testX.shape[0], look_back, testX.shape[1]))
    generator = make_model()
    discriminator = make_model()
    generator_optimizer = tf.keras.optimizers.Adam(1e-6)
    discriminator_optimizer = tf.keras.optimizers.Adam(1e-3)
    
    for i in range(10):
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_values = generator(trainX, training=True)
#             generator.fit(trainX, trainY, epochs=1, batch_size=1, verbose=2)
#             generated_values = generator(trainX)
#             decision = discriminator(trainX, training=True)
            discriminator.fit(trainX, trainY - generated_values)
            decision = generator(trainX)
        
            gen_loss = tf.sqrt(tf.losses.mean_squared_error(trainY, generated_values))
            disc_loss = tf.sqrt(tf.losses.mean_squared_error(trainY - generated_values, decision))

        gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
#         discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    testPredict = generator(testX, training = False)
    discPredict = discriminator(testX, training = False)
    #reshape testY to the shape wanted
    # Create a numpy array with testY 
    print(testY.shape, testPredict.shape)
    ge_score = round(math.sqrt(mean_squared_error([testY][0], testPredict)), 3)
    print("Gen Score: ", ge_score)
    gen_losses.append(ge_score)
    
    di_score = round(math.sqrt(mean_squared_error([testY][0], discPredict + testPredict)), 3)
    print("Disc Score: ", di_score)
    disc_losses.append(di_score)
#     print(testPredict.shape)
#     plt.plot(dataset[train_size + look_ahead + look_back: -1 ], label = "Ground")
    plt.plot([testY][0], label = "Test")
    plt.plot(testPredict, label = "Prediction")
    plt.plot((discPredict + testPredict), label = "Disc Predict")
    plt.title("Plot for " + str(look_ahead))
    plt.legend()
    plt.show()

plt.plot(gen_losses, label = "Gen Losses")
plt.plot(disc_losses, label = "Disc Losses")
plt.legend()
plt.show()


1314/1314 [==============================] - 12s 9ms/step - loss: 0.2885
